Denoising the lat-long detection data.

In [1]:
import numpy as np
import pandas as pd
from utils.map_display import *
from utils.data_ops import get_trajectory, read_and_init_detections_df

In [2]:
# Import the post filter M2 data. We only need the set of id_tracks
track_path = "../data/tracks_tagged_v1.csv"
track_data = pd.read_csv(track_path)
track_data.head(5)
track_ids = track_data["id_track"].unique()

In [3]:
# Import our detection data frame. (Post Processed)
detection_path = "../data/detections_tagged_cached.csv"
detection_data = pd.read_csv(detection_path)

In [4]:
# Use a simple convolution for rolling average
def convolve(input, filter):
    # Do some checks on the filter
    if not np.allclose(np.sum(filter), 1):
        raise RuntimeError("Entires in filter must sum to 1")
    if len(filter) % 2 == 0:
        raise RuntimeError("Filter Length must be odd")
    # add padding to the input array
    pad_length = len(filter) // 2
    pad_head = np.repeat(input[0], pad_length)
    pad_tail = np.repeat(input[-1], pad_length)
    # add the padding
    padded_input = np.hstack((pad_head, input, pad_tail))
    # Compute the colvolution
    return np.convolve(padded_input, filter, mode = "valid")

def generate_gaussian_kernel(length, std : float = 1):
    assert length % 2 == 1
    
    def gaussian_density(x):
        c = 1/(np.sqrt(2 * np.pi) * std)
        return c * np.exp(- 0.5 * (x**2) / (std ** 2))
    
    mid = length // 2
    result = [gaussian_density(x) for x in range(-mid, mid + 1)]
    return result / np.sum(result) # Normalize to make sure things sum to 1.


def denoise_filter(input, filter_length : int = 3, std : float = 2**31):
    """
    denoise filter via gaussian convolution

    Args:
        input: input lat long array
        filter_length: length of the smoothing filter. Defaults to 3.
        std: standard deviation of the gaussian kernel. Defaults to a simple average
    """
    filter = generate_gaussian_kernel(filter_length, std)
    
    lat, long = input[:,0], input[:, 1]
    lat_smoothed, long_smoothed = convolve(lat, filter), convolve(long, filter)
    return np.vstack((lat_smoothed, long_smoothed)).T
    
def denoise_filter_1d(input, filter_length : int = 3, std : float = 2**31):
    """
    A 1-D version of the denoise filter implementation above. Useful for 
    Pandas Apply
    """
    filter = generate_gaussian_kernel(filter_length, std)
    return convolve(input, filter)
    

In [7]:
# Get a random track id
id = np.random.choice(track_ids)
raw_detections = get_trajectory(detection_data, id)
location_center = np.mean(raw_detections, axis = 0)

In [8]:
import folium.plugins
# Left shows original data, Right shows denoised data
m = folium.plugins.DualMap(location = location_center, zoom_start = 13)
plot_trajectory(m.m1, raw_detections)
plot_trajectory(m.m2, denoise_filter(raw_detections, 7, 1.2))

In [7]:
# This is the filter used:
generate_gaussian_kernel(7, 1.2)

array([0.01464626, 0.08312087, 0.23555925, 0.33334724, 0.23555925,
       0.08312087, 0.01464626])

The following program applies the smoothing to all tracks within the detection data frame.

In [8]:
# Change the hyperparameter of smoothing here
def transform_fn(x:pd.Series):
    return denoise_filter_1d(x.to_numpy(), 7, 1.2)

def apply_denoise(dataframe):

    # First we need to sort the data frame with respect to track id and time
    out = dataframe.sort_values(by = ["id_track", "time"], ascending=True)
    # Group by id_track and extract the latitude and longitude columns
    g_latitude = out.groupby("id_track")["latitude"]
    g_longitude = out.groupby("id_track")["longitude"]
    # Write transofrmation to output
    out["latitude"] = g_latitude.transform(transform_fn)
    out["longitude"] = g_longitude.transform(transform_fn)
    return out

In [9]:
detection_data_mod = apply_denoise(detection_data) # About 10 seconds

In [18]:
# Check_correctness
id = np.random.choice(track_ids)
raw_detections = get_trajectory(detection_data, id)
location_center = np.mean(raw_detections, axis = 0)
smoothed_detections = get_trajectory(detection_data_mod, id)
# Left shows original data, Right shows denoised data
m = folium.plugins.DualMap(location = location_center, zoom_start = 13)
plot_trajectory(m.m1, raw_detections)
plot_trajectory(m.m2, smoothed_detections)

In [11]:
# Save transformed dataframe to file (40 seconds)
# detection_data_mod.to_csv("../data/detections_tagged_smoothed.csv", index=False)

In [ ]:
# Apply denoise to the radar detection file as well
# Requires pre_processing ~ 3 minutes
# detection_radar = read_and_init_detections_df("../data/detections_radar.csv", save=False)
# detection_radar_mod = apply_denoise(detection_radar)
# detection_radar_mod.to_csv("../data/detections_radar_smoothed.csv", index = False)

The imported detections file ../data/detections_tagged.csv requires preprocessing.          Please expect ~2 minutes to finish.


Preparing Trajectory Dataframe: 100%|██████████| 6756272/6756272 [02:11<00:00, 51332.28it/s]
